<a href="https://colab.research.google.com/github/Mohamed-Aziz-Mhadhbi/Drepanocytose/blob/main/Main_Drepanocytose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/InterxPim/Models.git

Cloning into 'Models'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 38 (delta 4), reused 31 (delta 4), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [ ]:
%cd Models/Drépanocytose

/content/Models/Drépanocytose


In [ ]:
!pip install -r requirement.txt

     |████████████████████████████████| 320.4 MB 24 kB/s 
     |████████████████████████████████| 13.1 MB 1.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 3.6 MB 41.3 MB/s 
     |████████████████████████████████| 6.8 MB 47.0 MB/s 
     |████████████████████████████████| 12.5 MB 25.7 MB/s 
     |████████████████████████████████| 26.6 MB 1.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 459 kB 42.1 MB/s 
     |████████████████████████████████| 2.9 MB 34.0 MB/s 
     |████████████████████████████████| 18.3 MB 491 kB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=2b25a72f1a56036507f5b4f1e287c29257a526b9ce4d795eff083eba8fa40c03
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
Successfully built gdown
  Attempting 

In [ ]:
import tensorflow as tf
import keras as K
tf.__version__, K.__version__

In [ ]:
import source.toolkit as tk
import source.dojo_tools as dj

from source.toolkit import list_channels
from source.toolkit import list_channels_df

# load main class object for monitoring blood cells
from source.SickleML_Monitor import CountAdheredBloodCells

# loading tools for extracting gdrive data
import source.load_data_tools as loading_tools

import os
import pandas as pd
import time
from tqdm import tqdm
import sys
import gdown
from natsort import natsorted

In [ ]:
os.makedirs(f'./Phase2_Pretrained-models/Resnet50/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Resnet50/') 

loading_tools.resnet50_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')


os.makedirs(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/', exist_ok = True)
os.chdir(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/') 

loading_tools.ce_jaccard_gdrive_()
    
os.chdir(f'../')
os.chdir(f'../')


In [ ]:
channel_path= './data/Manual-VS-AI/'
os.makedirs(channel_path, exist_ok=True)
os.chdir(channel_path)
loading_tools.laminin_channel_gdrive_()
os.chdir('../')
os.chdir('../')
channel_filenames  =  os.listdir(channel_path)

In [ ]:
print(f'Number of channels: {len(channel_filenames)}')
channel_filenames

In [ ]:
def load_ensembles(Phase1_path_model, Phase2_path_model):

    Phase1_path = './Phase1_Pretrained-models/' + Phase1_path_model + '/'# folder for Phase I 
    Phase1_filenames = natsorted(os.listdir(Phase1_path))
    Phase1_final_filenames = []
    for ii in range(len(Phase1_filenames)):
        if ii % 2 != 0:
            Phase1_final_filenames.append(Phase1_filenames[ii].replace('.json', ''))

    Phase2_path = './Phase2_Pretrained-models/' + Phase2_path_model + '/'# folder for Phase II
    Phase2_filenames = natsorted(os.listdir(Phase2_path))
    Phase2_final_filenames = []
    for ii in range(len(Phase2_filenames)):
        if ii % 2 != 0:
            Phase2_final_filenames.append(Phase2_filenames[ii].replace('.json',''))


    Phase1_ensemble = tk.load_zoo(Phase1_path, Phase1_final_filenames) # loading the Phase I ensemble (expect: 7)
    Phase2_ensemble = tk.load_zoo(Phase2_path, Phase2_final_filenames) # loading the Phase I ensemble (expect: 5)
    return Phase1_ensemble, Phase2_ensemble

def create_final_df(counts, times):
    counts_df = pd.DataFrame(counts)
    counts_df.columns = ['filename', 'def-sRBC', 'nondef-sRBC', 'Other']
    times_df = pd.DataFrame(times)
    times_df.columns = ['time_secs']
    final_df = pd.concat([counts_df, times_df], axis = 1)
    return final_df

In [ ]:
%%time 



counts, times = [], [] 
count_container, time_container = [], []

rbc_thres = [0.4]
wbc_thres = [0.4]
other_thres = [0.9]

Phase1_names, Phase2_names = 'ce-jaccard_encoder-decoder-net', 'Resnet50'

Phase1_ensemble, Phase2_ensemble = load_ensembles(Phase1_names, Phase2_names)
counts, times = [], [] 

for index, filenames in enumerate(channel_filenames):
    for rep in ((".png", ""), (".jpg", "")):
        clean_filename = filenames.replace(*rep)
    print('Analysis:', index, '| Channel:', clean_filename)
    print('==================================================================')
    channel = CountAdheredBloodCells(channel_path, filenames) # calling the class object
    # calling the function to output cell counts
    start = time.time()
    sRBC, WBC, Others, img_container, sRBC_container, WBC_container, Other_container = channel.call_pipeline(Phase1_ensemble, Phase2_ensemble, rbc_thres, wbc_thres, other_thres)
    end = time.time()
    run_time = end-start
            
    times.append([run_time])
    counts.append([filenames, sRBC, WBC, Others])
            
    final_df = create_final_df(counts,times)
   # final_df.to_csv(f'./AI-vs-Human_counts/{Phase1_name}_{Phase2_name}.csv', index = False)


In [ ]:
print("Here is the predicted counts ...")
final_df